# HW3

Task: Develop a Named Entity Recognition (NER) tool based on BERT model.

We download the libraries we need

In [1]:
!pip install transformers datasets seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_

We call ou libraries

In [2]:
import torch
import torch.nn as nn
import numpy as np

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_scheduler
from torch.optim import AdamW
from transformers import get_scheduler #to create a learning rate scheduler
from sklearn.metrics import classification_report #print precision, recall and f1-score
from tqdm import tqdm #to track training progress


from sklearn.utils.class_weight import compute_class_weight # to calculate the weight of weakly represented classes


Loading data from .txt

Extract tokkens and label


In [3]:
def read_ner_file(file_path):
    with open(file_path, encoding="utf-8") as f:
        lines = f.readlines()

    sentences, labels = [], []
    tokens, tags = [], []

    for line in lines:
        line = line.strip()
        if not line:
            if tokens:
                sentences.append(tokens)
                labels.append(tags)
                tokens, tags = [], []
        else:
            parts = line.split()
            if len(parts) == 2:
                word, tag = parts
                tokens.append(word)
                tags.append(tag)
            else:
                tokens.append(parts[0])
                tags.append("O")

    return sentences, labels


train_tokens, train_labels = read_ner_file("/content/train.txt")
valid_tokens, valid_labels = read_ner_file("/content/valid.txt")
test_tokens, test_labels = read_ner_file("/content/test.txt")


Dataset preparation:

We encode the tokens and labels with the SecBERT tokenizer.

As a reminder, label2id and id2label are there to translate our classes into numbers and move from one to the other.

In [4]:
model_name = "jackaduma/SecBERT" # I selected the first SecBERT model I found on huggingface: https://huggingface.co/jackaduma/SecBERT
tokenizer = AutoTokenizer.from_pretrained(model_name)

# label2id and id2label mappings
unique_labels = set(label for seq in train_labels for label in seq)
label2id = {label: idx for idx, label in enumerate(sorted(unique_labels))}
id2label = {idx: label for label, idx in label2id.items()}

#We show the labels to make sure they are not ugly and usable
print(f"{len(label2id)} labels : {label2id}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/378k [00:00<?, ?B/s]

27 labels : {'B-Area': 0, 'B-Exp': 1, 'B-Features': 2, 'B-HackOrg': 3, 'B-Idus': 4, 'B-OffAct': 5, 'B-Org': 6, 'B-Purp': 7, 'B-SamFile': 8, 'B-SecTeam': 9, 'B-Time': 10, 'B-Tool': 11, 'B-Way': 12, 'I-Area': 13, 'I-Exp': 14, 'I-Features': 15, 'I-HackOrg': 16, 'I-Idus': 17, 'I-OffAct': 18, 'I-Org': 19, 'I-Purp': 20, 'I-SamFile': 21, 'I-SecTeam': 22, 'I-Time': 23, 'I-Tool': 24, 'I-Way': 25, 'O': 26}


Here I just count the number of elements per class

In [5]:
from collections import Counter
import itertools

flat_labels = list(itertools.chain(*train_labels))
label_counts = Counter(flat_labels)
print(label_counts)


Counter({'O': 110593, 'B-HackOrg': 3418, 'B-Tool': 2449, 'B-Area': 2171, 'B-OffAct': 1412, 'I-Tool': 1386, 'B-Idus': 1349, 'B-Time': 1328, 'I-Purp': 1230, 'B-SamFile': 1221, 'I-Features': 1178, 'B-Org': 1113, 'B-Exp': 1068, 'B-SecTeam': 997, 'I-HackOrg': 977, 'I-Org': 958, 'I-OffAct': 851, 'B-Way': 827, 'I-Area': 816, 'B-Features': 812, 'I-Time': 794, 'I-Way': 784, 'B-Purp': 720, 'I-SamFile': 556, 'I-Exp': 538, 'I-Idus': 482, 'I-SecTeam': 473})


Some classes are very underrepresented compared to others. This is why it is necessary to penalize large and small classes differently.


# The model:

Composed of 3 parts:

-SecBERT

-A bidirectional BiLSTM layer,

-A linear classification layer.

In [6]:
class NERDataset(Dataset):
    def __init__(self, tokens, labels, tokenizer, label2id, max_len=128):
        self.tokens = tokens
        self.labels = labels
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_len = max_len

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        words = self.tokens[idx]
        tags = self.labels[idx]

        encoding = self.tokenizer(
            words,
            is_split_into_words=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )

        labels = [-100] * encoding.input_ids.shape[1]
        word_ids = encoding.word_ids(batch_index=0)

        for i, word_idx in enumerate(word_ids):
            if word_idx is not None:
                labels[i] = label2id[tags[word_idx]]

        return {
            "input_ids": encoding.input_ids.squeeze(),
            "attention_mask": encoding.attention_mask.squeeze(),
            "labels": torch.tensor(labels)
        }

train_dataset = NERDataset(train_tokens, train_labels, tokenizer, label2id)
valid_dataset = NERDataset(valid_tokens, valid_labels, tokenizer, label2id)
test_dataset = NERDataset(test_tokens, test_labels, tokenizer, label2id)


Model Training :

In [7]:
class SecBERT_BiLSTM(nn.Module):
    def __init__(self, model_name, num_labels, hidden_dim=256, dropout=0.3):
        super(SecBERT_BiLSTM, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.bilstm = nn.LSTM(
            input_size=self.bert.config.hidden_size,
            hidden_size=hidden_dim,
            num_layers=1,
            bidirectional=True,
            batch_first=True
        )
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(hidden_dim * 2, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state

        lstm_out, _ = self.bilstm(sequence_output)
        lstm_out = self.dropout(lstm_out)
        logits = self.classifier(lstm_out)

        loss = None
        if labels is not None:
          # This is where classes are penalized differently depending on their number
          all_labels_flat = [label2id[l] for seq in train_labels for l in seq]
          class_weights = compute_class_weight('balanced', classes=np.array(list(label2id.values())), y=all_labels_flat)
          class_weights = torch.tensor(np.sqrt(class_weights), dtype=torch.float).to(device)


          loss_fct = nn.CrossEntropyLoss(weight=class_weights, ignore_index=-100)
          loss = loss_fct(logits.view(-1, logits.shape[-1]), labels.view(-1))

        return (loss, logits) if loss is not None else logits


Evaluate the model at each Epoch to check how the model learns.

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SecBERT_BiLSTM(model_name, num_labels=len(label2id)).to(device)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8)

optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 5

scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_loader) * num_epochs,
)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        loss, _ = model(**batch)

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} loss: {total_loss:.4f}")

    # model evaluation
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in valid_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs if isinstance(outputs, torch.Tensor) else outputs[1]

            predictions = torch.argmax(logits, dim=-1)

            for i in range(labels.shape[0]):
                true = labels[i]
                pred = predictions[i]
                for t, p in zip(true, pred):
                    if t != -100:
                        all_labels.append(id2label[t.item()])
                        all_preds.append(id2label[p.item()])

    print(classification_report(all_labels, all_preds, digits=4))


model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Epoch 1: 100%|██████████| 657/657 [01:16<00:00,  8.54it/s]


Epoch 1 loss: 1128.8474
              precision    recall  f1-score   support

      B-Area     0.5768    0.8580    0.6898       162
       B-Exp     0.8525    0.9813    0.9124       589
  B-Features     0.5138    0.5657    0.5385        99
   B-HackOrg     0.5166    0.7513    0.6122       599
      B-Idus     0.2865    0.8966    0.4342       116
    B-OffAct     0.3241    0.5833    0.4167       120
       B-Org     0.4286    0.0923    0.1519       130
      B-Purp     0.4000    0.3218    0.3567        87
   B-SamFile     0.4457    0.7229    0.5514       397
   B-SecTeam     0.5477    0.7110    0.6188       218
      B-Time     0.6434    0.8058    0.7155       206
      B-Tool     0.3936    0.5032    0.4417       467
       B-Way     0.4759    0.6371    0.5448       124
      I-Area     0.4884    0.8400    0.6176        50
       I-Exp     0.3285    1.0000    0.4945        45
  I-Features     0.3476    0.6842    0.4610        95
   I-HackOrg     0.3178    0.4870    0.3846       154
   

Epoch 2: 100%|██████████| 657/657 [01:17<00:00,  8.52it/s]


Epoch 2 loss: 501.3032
              precision    recall  f1-score   support

      B-Area     0.7115    0.9136    0.8000       162
       B-Exp     0.8947    0.9813    0.9360       589
  B-Features     0.5455    0.9091    0.6818        99
   B-HackOrg     0.5444    0.8497    0.6636       599
      B-Idus     0.6222    0.7241    0.6693       116
    B-OffAct     0.3932    0.6750    0.4969       120
       B-Org     0.3651    0.5308    0.4326       130
      B-Purp     0.3163    0.7126    0.4382        87
   B-SamFile     0.6038    0.7985    0.6876       397
   B-SecTeam     0.6392    0.7477    0.6892       218
      B-Time     0.6992    0.8689    0.7749       206
      B-Tool     0.4815    0.6124    0.5391       467
       B-Way     0.6069    0.7097    0.6543       124
      I-Area     0.5000    0.9000    0.6429        50
       I-Exp     0.5556    1.0000    0.7143        45
  I-Features     0.4314    0.9263    0.5886        95
   I-HackOrg     0.3773    0.6688    0.4824       154
    

...

Epoch 3 loss: 316.1719
              precision    recall  f1-score   support

      B-Area     0.7353    0.9259    0.8197       162
       B-Exp     0.9107    0.9864    0.9470       589
  B-Features     0.5497    0.9495    0.6963        99
   B-HackOrg     0.5616    0.8598    0.6794       599
      B-Idus     0.6620    0.8103    0.7287       116
    B-OffAct     0.3700    0.7000    0.4841       120
       B-Org     0.4671    0.5462    0.5035       130
      B-Purp     0.3013    0.7931    0.4367        87
   B-SamFile     0.7224    0.7733    0.7470       397
   B-SecTeam     0.7106    0.7661    0.7373       218
      B-Time     0.7595    0.8738    0.8126       206
      B-Tool     0.4554    0.7537    0.5677       467
       B-Way     0.5238    0.7984    0.6326       124
      I-Area     0.4639    0.9000    0.6122        50
       I-Exp     0.6250    1.0000    0.7692        45
  I-Features     0.4034    0.9895    0.5732        95
   I-HackOrg     0.3889    0.7273    0.5068       154
    

Epoch 4: 100%|██████████| 657/657 [01:16<00:00,  8.56it/s]


Epoch 4 loss: 231.8395
              precision    recall  f1-score   support

      B-Area     0.7238    0.9383    0.8172       162
       B-Exp     0.9121    0.9864    0.9478       589
  B-Features     0.6065    0.9495    0.7402        99
   B-HackOrg     0.5862    0.8681    0.6999       599
      B-Idus     0.6182    0.8793    0.7260       116
    B-OffAct     0.4372    0.7250    0.5455       120
       B-Org     0.5070    0.5538    0.5294       130
      B-Purp     0.3393    0.8736    0.4887        87
   B-SamFile     0.6157    0.8715    0.7216       397
   B-SecTeam     0.7857    0.7569    0.7710       218
      B-Time     0.7788    0.8544    0.8148       206
      B-Tool     0.5096    0.6852    0.5845       467
       B-Way     0.5722    0.8306    0.6776       124
      I-Area     0.4839    0.9000    0.6294        50
       I-Exp     0.6250    1.0000    0.7692        45
  I-Features     0.4519    0.9895    0.6205        95
   I-HackOrg     0.4167    0.7143    0.5263       154
    

Epoch 5: 100%|██████████| 657/657 [01:16<00:00,  8.56it/s]


Epoch 5 loss: 194.0070
              precision    recall  f1-score   support

      B-Area     0.7600    0.9383    0.8398       162
       B-Exp     0.9179    0.9864    0.9509       589
  B-Features     0.6139    0.9798    0.7549        99
   B-HackOrg     0.5991    0.8731    0.7106       599
      B-Idus     0.7063    0.8707    0.7799       116
    B-OffAct     0.4508    0.7250    0.5559       120
       B-Org     0.5200    0.6000    0.5571       130
      B-Purp     0.3585    0.8736    0.5084        87
   B-SamFile     0.6413    0.8690    0.7380       397
   B-SecTeam     0.8000    0.7523    0.7754       218
      B-Time     0.7729    0.8592    0.8138       206
      B-Tool     0.5129    0.6809    0.5851       467
       B-Way     0.5492    0.8548    0.6688       124
      I-Area     0.4839    0.9000    0.6294        50
       I-Exp     0.6164    1.0000    0.7627        45
  I-Features     0.4563    0.9895    0.6246        95
   I-HackOrg     0.4264    0.7143    0.5340       154
    

Predictions on test dataset

In [9]:
test_loader = DataLoader(test_dataset, batch_size=8)

model.eval()
all_test_preds = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = logits if isinstance(logits, torch.Tensor) else logits[1]
        predictions = torch.argmax(logits, dim=-1)

        for pred, mask in zip(predictions, batch["attention_mask"]):
            tokens_pred = []
            for p, m in zip(pred, mask):
                if m == 1:
                    tokens_pred.append(id2label[p.item()])
            all_test_preds.append(tokens_pred)


Testing: 100%|██████████| 83/83 [00:03<00:00, 24.04it/s]


Exporting predictions

In [10]:
input_test_file = "/content/test.txt"
output_prediction_file = "test_predictions.txt"

with open(input_test_file, "r", encoding="utf-8") as f:
    test_lines = f.readlines()

with open(output_prediction_file, "w", encoding="utf-8") as f:
    pred_idx = 0
    for line in test_lines:
        line = line.strip()
        if line == "":
            f.write("\n")
        else:
            token = line.split()[0]
            if len(all_test_preds) > 0 and len(all_test_preds[-1]) == 0:
              all_test_preds = all_test_preds[:-1]

            flat_preds = [label for sent in all_test_preds for label in sent]
            label = flat_preds[pred_idx] if pred_idx < len(flat_preds) else "O"

            f.write(f"{token} {label}\n")
            pred_idx += 1

print(f" file completed : {output_prediction_file}")


 file completed : test_predictions.txt
